In [1]:
import pickle
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score



In [2]:
df = pd.read_csv('.//LRBF-2/LRBF-2/data/input_output.csv')




In [3]:
df

,name,Npoints,Nboundary,ri,re,xcenter,ycenter,number,c,eig1,eig2,eig3,eig4
0,1,100,100,0.200000,10,7.340740,-4.935545,20,0.2,0.086835,0.205047,0.217693,0.394798
1,2,100,100,0.209309,10,-8.633278,-1.806293,20,0.2,0.089757,0.208017,0.219528,0.401879
2,3,100,100,0.218619,10,-2.527040,-7.758822,20,0.2,0.088745,0.211512,0.219823,0.393512
3,4,100,100,0.227928,10,9.139652,-0.637795,20,0.2,0.090136,0.205796,0.219625,0.393025
4,5,100,100,0.237237,10,-7.321746,-4.218163,20,0.2,0.088790,0.205965,0.219218,0.392779
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,100,100,9.462763,10,-0.236382,-0.339796,20,0.2,6.288588,6.333078,6.440969,6.580474
996,997,100,100,9.472072,10,0.361274,-0.020281,20,0.2,6.736554,6.910631,7.249757,7.427355
997,998,100,100,9.481381,10,0.422731,0.068112,20,0.2,6.235476,6.365920,6.499170,6.727749
998,999,100,100,9.490691,10,-0.037546,-0.398810,20,0.2,6.172188,6.383114,6.457401,6.527400


In [4]:
X = df[["xcenter", "ycenter", "ri"]]
Y = df[["eig1", "eig2", "eig3", "eig4"]]

In [5]:
X

,xcenter,ycenter,ri
0,7.340740,-4.935545,0.200000
1,-8.633278,-1.806293,0.209309
2,-2.527040,-7.758822,0.218619
3,9.139652,-0.637795,0.227928
4,-7.321746,-4.218163,0.237237
...,...,...,...
995,-0.236382,-0.339796,9.462763
996,0.361274,-0.020281,9.472072
997,0.422731,0.068112,9.481381
998,-0.037546,-0.398810,9.490691


In [6]:
Y

,eig1,eig2,eig3,eig4
0,0.086835,0.205047,0.217693,0.394798
1,0.089757,0.208017,0.219528,0.401879
2,0.088745,0.211512,0.219823,0.393512
3,0.090136,0.205796,0.219625,0.393025
4,0.088790,0.205965,0.219218,0.392779
...,...,...,...,...
995,6.288588,6.333078,6.440969,6.580474
996,6.736554,6.910631,7.249757,7.427355
997,6.235476,6.365920,6.499170,6.727749
998,6.172188,6.383114,6.457401,6.527400


In [7]:
X_train, X_valid_test, Y_train, Y_valid_test = train_test_split(
    X, Y, test_size=0.3, random_state=25
)

In [8]:
X_valid, X_test, Y_valid, Y_test = train_test_split(X_valid_test, Y_valid_test, test_size=0.33, random_state=25)

In [10]:
r2_max = 0.0
for n_layer in range(3, 8):
    for n_node in range(15, 20):
        mlp = MLPRegressor(
            hidden_layer_sizes=tuple([n_node for i in range(n_layer)]),  
            activation="relu",
            solver="adam",
            alpha=0.0001,
            learning_rate_init=0.001,
            max_iter=5000,
            validation_fraction=0.15,
            random_state=25
        )
        
        model = Pipeline([
            ("x_scaler", StandardScaler()),
            ("reg", TransformedTargetRegressor(
                regressor=mlp,
            ))
        ])
        
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_valid)
        
        mse = mean_squared_error(Y_valid, Y_pred)
        r2 = r2_score(Y_valid, Y_pred)
        
        
        file_name = f'./models/ann_{n_layer}layer_{n_node}node_{r2:.4f}r2.sav'
        
        if (r2>0.99 and mse<0.01):
            print(f"mse: {mse},   r2: {r2},   n_node: {n_node},   n_layer: {n_layer}")
            pickle.dump(model, open(file_name, 'wb'))
            
            if r2 > r2_max:
                r2_max, layer_max, node_max = r2, n_layer, n_node
print(f'r2_max: {r2_max}, n_layer: {layer_max}, n_node: {node_max}')

mse: 0.009358168851163428,   r2: 0.9949389378148906,   n_node: 19,   n_layer: 5
mse: 0.008883677193415975,   r2: 0.9951847836582797,   n_node: 18,   n_layer: 6
mse: 0.008349809366971562,   r2: 0.995484953538125,   n_node: 16,   n_layer: 7
r2_max: 0.995484953538125, n_layer: 7, n_node: 16


In [11]:
load_model = pickle.load(open("./models/ann_7layer_16node_0.9955r2.sav", 'rb'))

In [12]:
r2_score(Y_test, load_model.predict(X_test))

0.9968082649738984

In [13]:
mean_squared_error(Y_test, load_model.predict(X_test))

0.006466812432126698

In [16]:
X_test

,xcenter,ycenter,ri
915,0.280687,-1.134137,8.718018
103,7.552113,-1.303594,1.158859
194,-2.665103,5.315154,2.006006
725,1.723983,-0.319559,6.949249
100,-0.669561,5.683575,1.130931
...,...,...,...
183,0.472718,-3.957550,1.903604
779,1.300793,1.318097,7.451952
241,-1.873780,-4.546715,2.443544
416,1.514364,-3.952955,4.072673
